In [1]:
import os
import json
import numpy as np
import pandas as pd

In [2]:
def get_avg_std(metric_list, percent=100, std_flag=False):
    mean_val = np.mean(metric_list)
    std_val = np.std(metric_list)
    if std_flag:
        return f"{mean_val*percent:.2f}±{std_val:.2f}"
    else:
        return np.round(mean_val*percent, 2)


def summarize_results(json_folder, std_flag=False):
    metrics_ls, metrics_ls_multi_hop = [], []

    for filename in sorted(os.listdir(json_folder)):
        if filename.endswith('.json') and 'multi_turn' not in filename:
            filepath = os.path.join(json_folder, filename)
            with open(filepath, 'r') as file:
                metrics = json.load(file)
                parts = filename.split('_')
                domain = parts[0]
                topic = ' '.join(parts[1:-1])  
                edit_method = parts[-1].replace('.json', '')  

                results = {
                    "domain": domain,
                    "topic": topic,
                    "edit_method": edit_method,
                    "efficacy_pre": get_avg_std([e['pre']['edit_acc'][0] for e in metrics]),
                    "efficacy_post": get_avg_std([e['post']['edit_acc'][0] for e in metrics]),
                    "rephrase_pre": get_avg_std([e['pre']['rephrase_acc'][0] for e in metrics]),
                    "rephrase_post": get_avg_std([e['post']['rephrase_acc'][0] for e in metrics]),
                    "yes_pre": get_avg_std([e['pre']['yes_questions']['yes_acc'][0] for e in metrics]),
                    "yes_post": get_avg_std([e['post']['yes_questions']['yes_acc'][0] for e in metrics]),
                    "no_pre": get_avg_std([e['pre']['no_questions']['no_acc'][0] for e in metrics]),
                    "no_post": get_avg_std([e['post']['no_questions']['no_acc'][0] for e in metrics]),
                    "mc_pre": get_avg_std([e['pre']['multiple_choice_questions']['multiple_choice_acc'][0] for e in metrics]),
                    "mc_post": get_avg_std([e['post']['multiple_choice_questions']['multiple_choice_acc'][0] for e in metrics]),
                    "reversed_pre": get_avg_std([e['pre']['reversed_relation_questions']['reversed_relation_acc'][0] for e in metrics]),
                    "reversed_post": get_avg_std([e['post']['reversed_relation_questions']['reversed_relation_acc'][0] for e in metrics]),
                    "locality_post": get_avg_std([e['post']['locality'][f'locality_acc'][0] for e in metrics]),
                }

                results_multi_hop = {
                    "domain": domain,
                    "topic": topic,
                    "edit_method": edit_method,
                    "questions_2hop_pre": get_avg_std([e['pre']['questions_2hop']['2hop_acc'][0] for e in metrics]),
                    "questions_2hop_post": get_avg_std([e['post']['questions_2hop']['2hop_acc'][0] for e in metrics]),
                    "questions_3hop_pre": get_avg_std([e['pre']['questions_3hop']['3hop_acc'][0] for e in metrics]),
                    "questions_3hop_post": get_avg_std([e['post']['questions_3hop']['3hop_acc'][0] for e in metrics]),
                    "questions_4hop_pre": get_avg_std([e['pre']['questions_4hop']['4hop_acc'][0] for e in metrics]),
                    "questions_4hop_post": get_avg_std([e['post']['questions_4hop']['4hop_acc'][0] for e in metrics]),
                    "questions_5hop_pre": get_avg_std([e['pre']['questions_5hop']['5hop_acc'][0] for e in metrics]),
                    "questions_5hop_post": get_avg_std([e['post']['questions_5hop']['5hop_acc'][0] for e in metrics]),
                    "questions_6hop_pre": get_avg_std([e['pre']['questions_6hop']['6hop_acc'][0] for e in metrics]),
                    "questions_6hop_post": get_avg_std([e['post']['questions_6hop']['6hop_acc'][0] for e in metrics]),
                }
                metrics_ls.append(results)
                metrics_ls_multi_hop.append(results_multi_hop)
    df = pd.DataFrame(metrics_ls)
    df_multi_hop = pd.DataFrame(metrics_ls_multi_hop)
    df = df.set_index('edit_method').loc[edit_method_order_ls].reset_index()
    df_multi_hop = df_multi_hop.set_index('edit_method').loc[edit_method_order_ls].reset_index()
    return df, df_multi_hop

model_id_ls = ['meta-llama/Meta-Llama-3-8B-Instruct', 'mistralai/Mistral-7B-Instruct-v0.3', 'meta-llama/Llama-2-7b-chat-hf']
model_id_format_ls = [e.split('/')[-1].replace('-', '_').lower() for e in model_id_ls]
model_name_ls = ["Llama2-7B", "Llama3-8B", "Mistral-v0.3-7B"]
edit_method_order_ls = ['FT-L', 'FT-M', 'MEMIT', 'ROME', 'LoRA', 'ICL', 'GRACE']

In [3]:
df_llama2, df_multi_hop_llama2 = summarize_results("../results/llama_2_7b_chat_hf")
df_llama3, df_multi_hop_llama3 = summarize_results("../results/meta_llama_3_8b_instruct")
df_mistral, df_multi_hop_mistral = summarize_results("../results/mistral_7b_instruct_v0.3")

In [4]:
df_llama3

,edit_method,domain,topic,efficacy_pre,efficacy_post,rephrase_pre,rephrase_post,yes_pre,yes_post,no_pre,no_post,mc_pre,mc_post,reversed_pre,reversed_post,locality_post
0,FT-L,art,sculpture,0.0,47.00,12.00,59.00,61.00,46.00,26.00,10.00,34.00,33.00,6.00,0.00,16.00
1,FT-L,business,brand,0.0,56.00,12.00,52.00,62.00,51.00,22.00,14.00,27.00,28.00,14.00,5.00,15.00
2,FT-L,business,corporation,0.0,37.78,5.56,34.44,62.22,45.56,14.44,6.67,27.78,32.22,4.44,0.00,26.67
3,FT-L,business,industry,0.0,52.04,18.37,53.06,88.78,41.84,26.53,7.14,31.63,38.78,17.35,13.27,12.24
4,FT-L,entertainment,anime,0.0,42.00,7.00,48.00,63.00,30.00,22.00,10.00,24.00,25.00,2.00,0.00,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,GRACE,places,country,0.0,100.00,19.00,2.00,61.00,10.00,42.00,32.00,51.00,1.00,51.00,21.00,91.00
178,GRACE,places,landmark,0.0,100.00,12.00,3.00,60.00,8.00,41.00,21.00,43.00,1.00,19.00,3.00,40.00
179,GRACE,technology,database,0.0,100.00,8.54,2.44,73.17,23.17,29.27,23.17,39.02,2.44,4.88,1.22,24.39
180,GRACE,technology,programming language,0.0,100.00,19.00,2.00,61.00,15.00,36.00,13.00,27.00,3.00,19.00,4.00,40.00


In [5]:
df_multi_hop_llama3

,edit_method,domain,topic,questions_2hop_pre,questions_2hop_post,questions_3hop_pre,questions_3hop_post,questions_4hop_pre,questions_4hop_post,questions_5hop_pre,questions_5hop_post,questions_6hop_pre,questions_6hop_post
0,FT-L,art,sculpture,33.00,17.00,32.00,13.00,34.00,19.00,34.00,21.00,45.00,32.00
1,FT-L,business,brand,37.00,19.00,32.00,15.00,31.00,19.00,26.00,18.00,28.00,19.00
2,FT-L,business,corporation,26.67,8.89,33.33,10.00,12.22,5.56,18.89,11.11,25.56,15.56
3,FT-L,business,industry,52.04,13.27,35.71,14.29,23.47,12.24,27.55,17.35,29.59,17.35
4,FT-L,entertainment,anime,14.00,4.00,25.00,5.00,26.00,7.00,24.00,13.00,29.00,18.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,GRACE,places,country,29.00,1.00,24.00,3.00,29.00,2.00,20.00,1.00,22.00,0.00
178,GRACE,places,landmark,45.00,3.00,37.00,3.00,36.00,2.00,34.00,0.00,37.00,1.00
179,GRACE,technology,database,45.12,0.00,28.05,0.00,20.73,0.00,23.17,0.00,28.05,1.22
180,GRACE,technology,programming language,41.00,4.00,28.00,0.00,25.00,0.00,32.00,0.00,28.00,1.00
